Scroll a specific element?


In [5]:
function scrollClient(selector, up = false) {
    // scroll to bottom of messages
    return client
        .execute((selector, up = false) => {
            if (selector === '' || selector === false) {
                window.scroll(window.scrollX, window.scrollY + (up ? -100000 : 100000));
                return;
            }
            var people = document.evaluate(
                selector,
                document, null,
                XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
            people.scrollTop = people.scrollTop + (up ? -100000 : 100000);
        }, selector, up)
};
module.exports = scrollClient;
scrollClient;


[Function: scrollClient]

Get all elements XPath?



In [3]:
function getAllXPathBrowser(selector) {
    var getArrayXPath = (selector, ctx, index) => {
        if (typeof ctx == 'undefined') {
            ctx = document;
        }
        var iterator = document.evaluate(
            selector.replace('$i', index),
            ctx, null,
            XPathResult.ORDERED_NODE_ITERATOR_TYPE, null);
        var co = [];
        var m;
        while ((m = iterator.iterateNext()) && co.push(m.nodeValue || m)) ;
        return co;
    };

    var getObjectXPath = (object, ctx, index) => {
        var result = {};
        // TODO: parse object
        for (var p in object) {
            if (object.hasOwnProperty(p)) {
                result[p] = resolveXPathQueries(object[p], ctx, index);
            }
        }
        return result;
    }

    var resolveXPathQueries = (selector, ctx, index) => {
        var inputArr = selector;
        if (!Array.isArray(inputArr)) {
            inputArr = [inputArr];
        }
        // parse array of string and objects
        var result = [], previousResults;
        for (var i = 0; i < inputArr.length; i++) {
            if (typeof inputArr[i] == 'string') {
                if (typeof previousResults != 'undefined') {
                    result = result.concat(previousResults);
                }
                // store in case the next query is a mapped object
                previousResults = getArrayXPath(inputArr[i], ctx, index);
            } else if (typeof inputArr[i] == 'object') {
                if (typeof previousResults !== 'undefined') {
                    previousResults = previousResults.map((p, j) =>
                        getObjectXPath(inputArr[i], p, j));
                    result = result.concat(previousResults);
                    previousResults = undefined;
                } else {
                    result[result.length] = getObjectXPath(inputArr[i], ctx, index);
                }
            } else if (Array.isArray(inputArr[i])) {
                result = result.concat(resolveXPathQueries(inputArr[i], ctx));
            } else if (typeof inputArr[i] == 'function') {
                // TODO: what to do here?
            }
        }
        if (typeof previousResults != 'undefined') {
            result = result.concat(previousResults);
        }

        if (!Array.isArray(selector) && result.length == 1) {
            return result[0];
        }
        return result;
    };
    return resolveXPathQueries(selector);
}

// TODO: better way to do this for selenium versus browser?
function getAllXPath(selector) {
    return client.execute(getAllXPathBrowser, selector)
        .then(r => r.value)
        .catch(e => console.log(e));
}

if (typeof client.getAllXPath == 'undefined') {
    client.addCommand('getAllXPath', getAllXPath);
}
module.exports = getAllXPath;


[Function: getAllXPath]

Get all elements until?



In [7]:
var importer = require('../Core');
var {
    getAllXPath, scrollClient
} = importer.import([
    'all elements xpath',
    'scroll specific element'
], {client})

function getAllUntil(scrollableSelector,
                     dataSelector,
                     set = [],
                     compare = (a, b) => a === b,
                     cb = (i) => i < 3,
                     up = false,
                     i = 0) {
    return getAllXPath(dataSelector)
        .then(r => {
            var newPosts = ((typeof r === 'string' ? [r] : r) || [])
                .filter(e => set
                    .filter(m => compare(e, m)).length === 0);
            set = newPosts.concat(set);
            return newPosts.length > 0
                ? scrollClient(scrollableSelector, up)
                    .pause(4000)
                    .then(() => cb(i)
                        ? getAllUntil(scrollableSelector, dataSelector, set, compare, cb, up, i + 1)
                        : set)
                : Promise.resolve(set);
        })
        .catch(e => console.log(e))
};
if (typeof client.getAllUntil == 'undefined') {
    client.addCommand('getAllUntil', getAllUntil);
}
module.exports = getAllUntil;




[Function: getAllUntil]

click spa link?


In [ ]:
function clickSpa(profile) {
    return client
        .getUrl()
        .then(url => url.indexOf(profile) > -1
              ? []
              : client
             .execute(url => {
                var a = document.createElement('a');
                a.setAttribute('href', url);
                a.setAttribute('class', 'spa');
                a.onclick = () => setTimeout(() => document.body.removeChild(a), 500)
                a.href = url;
                a.style.position = 'absolute';
                a.style.zIndex = 4294967295;
                a.style.top = 0;
                a.style.left = 0;
                a.style.bottom = 0;
                a.style.right = 0;
                document.body.appendChild(a);
            }, profile)
        .click('a.spa[href*="' + profile + '"]')
        .pause(1000))
        .catch(e => console.log(e))
};
if (typeof client.clickSpa == 'undefined') {
    client.addCommand('clickSpa', clickSpa);
}
module.exports = clickSpa;
